In [11]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    Article
)
from paperview.retrieval import pdf_extraction
from paperview.retrieval import process_xml
import tempfile
from io import StringIO
from PIL import Image
import seaborn as sns
import pandas as pd
from io import StringIO

import pdfplumber

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
# url = 'https://www.biorxiv.org/content/10.1101/456574v1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.21.521407v1?rss=1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.09.12.507457v1.full'
# url = 'https://www.biorxiv.org/content/10.1101/386375v1.full'
# url = 'https://www.biorxiv.org/content/10.1101/2022.07.20.500837v1?rss=1'
# url = 'https://www.biorxiv.org/content/10.1101/180125v2.full'


In [3]:
article = Article.from_content_page_url(
    url, 
    prog_bar=True, 
    extract_images=False)

100%|██████████| 36/36 [00:03<00:00,  9.92it/s]


In [4]:
article.get_overview().display()

In [ ]:
article.article_detail

In [15]:
xmlstr = article.retrieve_jats_xml()

In [16]:
text = process_xml.xmlstr_to_dict(xmlstr)
df = pd.DataFrame(text['body_sections'])
sections = df.query('level == 2').set_index('title')['id'].to_dict()
texts = []
figures = []
tables = []
for sec_title, sec_id in sections.items():
    subsection_idx = df.id.str.startswith(sec_id)
    subsections = df.loc[subsection_idx]
    
    for subsec_id, subsection in subsections.set_index('id').iterrows():
        contents = subsection['contents']
        for content in contents:
            if content.get('tag') == 'p':
                texts.append({
                    'section': sec_title,
                    'id': content['id'],
                    'text': content['text']
                })
            elif content.get('tag') == 'fig':
                figures.append({
                    'section': sec_title,
                    'id': content['id'],
                    'caption': content['caption'],
                    'label': content['label'],
                    'xref_id': content.get('xref_id')
                })
            elif content.get('tag') == 'table':
                tables.append({
                    'section': sec_title,
                    'id': content['id'],
                    'caption': content['caption'],
                    'label': content['label'],
                    'xref_id': content.get('xref_id')
                })

texts = pd.DataFrame(texts)
figures = pd.DataFrame(figures)
tables = pd.DataFrame(tables)

In [116]:
pd.DataFrame(texts)

,section,id,text
0,Graphical abstract,2.2.1,"Graphical summary of the design, results, and ..."
1,Introduction,3.1.1,Aging is the single greatest risk factor for n...
2,Introduction,3.1.2,Quantitative outcome measures to assess the mu...
3,Introduction,3.1.3,The Canine Frailty Index (CFI) was developed t...
4,Introduction,3.1.4,"In human clinical trials, patient-reported out..."
5,Introduction,3.1.5,In addition to potentially serving as outcome ...
6,Introduction,3.1.6,While both the CFI and the VetMetrica HRQL ins...
7,Materials and Methods,3.2.1.1,The study was performed according to VICH-GL9 ...
8,Materials and Methods,3.2.2.1,"This study was a multi-center, cross-sectional..."
9,Materials and Methods,3.2.2.3,Dogs were excluded from the study if they had ...


In [126]:
pd.DataFrame(figures)

,section,id,caption,label,xref_id
0,Graphical abstract,2.2.2,,,ufig1
1,Materials and Methods,3.2.2.2,Five hundred and eighty-six dogs were screened...,Fig. 1,fig1
2,Materials and Methods,3.2.6.8,DAG (directed acyclic graph) represents the me...,Fig. 2,fig2
3,Results,3.3.2.2.2,Canine frailty index scores in the different a...,Fig. 3,fig3
4,Results,3.3.2.3.2,Canine frailty index (CFI) scores in the diffe...,Fig. 4,fig4
5,Results,3.3.3.2.2,a. Total Health related quality of life (HRQL)...,Fig. 5,fig5
6,Results,3.3.3.3.2,a. Total Health Related Quality of Life (HRQL)...,Fig. 6,fig6
7,Results,3.3.4.2,a. Scatter plots of total Health Related Quali...,Fig. 7,fig7
8,Results,3.3.5.2,The most common Canine Frailty Index (CFI) Fea...,Fig. 8,fig8


In [127]:
pd.DataFrame(tables)

,section,id,caption,label,xref_id
0,Materials and Methods,3.2.3.2,Schedule of events,Table 1,tbl1
1,Results,3.3.1.2,"Demographics of study population with age, sex...",Table 2,tbl2
2,Results,3.3.2.1.2,Canine Frailty Index (CFI) and Health Related ...,Table 3,tbl3
3,Results,3.3.2.3.3,Linear contrast estimates comparing difference...,Table 4,tbl4


In [124]:
df.loc[16,'contents']

[{'tag': 'p',
  'text': 'Old dogs had significantly higher CFI scores compared to young dogs. A Kruskal-Wallis test showed a significant difference in CFI scores across age-size groups (H(3)= 182.87, p < 0.001) (Table 3). Pairwise Wilcoxon-rank sum tests demonstrated that CFI scores were significantly higher in old dogs compared to young dogs, in both large (p-adjusted < 0.001) and small (p-adjusted < 0.001) dogs (Fig. 3).',
  'id': '3.3.2.2.1'},
 {'tag': 'fig',
  'caption': 'Canine frailty index scores in the different age-size groups (young and small, old and small, young and large, old and large) are shown with box and whisker plots. Young dogs were ≥ 2 and <6 years, old dogs were ≥7 years, small dogs were < 25 lbs and large dogs were > 50 lbs. The median is the line in the center of the box and the X represents the mean. The vertical line at the bottom of the box denotes the first quartile and the vertical line extending from the top of the box denotes the third quartile. Pairwise 

In [103]:
pd.DataFrame(texts)

,id,text
0,3.3.1.1,"Following screening, 451 eligible adult dogs w..."
1,3.3.1.3,"Of the total population (n = 451), 450 dogs ha..."
2,3.3.2.1.1,The distribution of CFI scores was wider in ol...
3,3.3.2.2.1,Old dogs had significantly higher CFI scores c...
4,3.3.2.3.1,Scatter plots showed trends of increasing CFI ...
5,3.3.3.1.1,Older dogs had a wider distribution (higher IQ...
6,3.3.3.2.1,Older dogs had significantly lower total HRQL ...
7,3.3.3.3.1,Scatter plots showed trends of decreasing tota...
8,3.3.3.4.1,Likelihood ratio tests of multiple linear regr...
9,3.3.4.1,CFI score and total HRQL score were inversely ...


In [91]:
subsections.set_index('id').loc['3.3.1', 'contents']

[{'tag': 'p',
  'text': 'Following screening, 451 eligible adult dogs were enrolled of which 46.3% were female and 53.7% male. The mean age of dogs was 7.72 years (range = 2.0 to 17.7, SD = 3.8). Mean weight was 24.0 kg (range 1.3-80, SD 15). The majority (91.1%) of dogs were neutered. The evaluable population consisted of 43.6% mixed breeds and 56.4% purebred dogs. The demographics of the evaluated subjects in each of the age and size groups are presented in Table 2. A table of enrolled breeds <11.3 kg / <25 lbs (small dogs) or > 22.7 kg / >50 lbs (large dogs) is presented in Supp Table 1, Table 2, respectively.',
  'id': '3.3.1.1'},
 {'tag': 'table',
  'xref_id': 'tbl2',
  'xref_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles//table/tbl2',
  'label': 'Table 2',
  'caption': 'Demographics of study population with age, sex, and breed information in 4 age-size groups (young and small, old and small, young and large, old and large)',
  'footer': '',
  'media': [],
  'graphics': ['500746

In [87]:
subsection

implicit      False
level             2
label              
title       Results
contents         []
caption            
tag             sec
Name: 3.3, dtype: object

In [72]:
sections

{'Graphical abstract': '2.2',
 'Introduction': '3.1',
 'Materials and Methods': '3.2',
 'Results': '3.3',
 'Discussion': '3.4',
 'Conclusion': '3.5',
 'Supporting information': '3.6'}

In [35]:
df.loc['3.3.2.1', 'contents']

[{'tag': 'p',
  'text': 'The distribution of CFI scores was wider in old dogs compared to young dogs. The distribution of the CFI scores in each age-size group was non-normal with positive skew. CFI scores in the evaluable population of dogs (n = 450) ranged from 0.0 to 0.59, with a median of 0.03, and IQR of 0.09. IQRs of CFI scores in old dog groups were larger than that of young dog groups (Table 3).',
  'id': '3.3.2.1.1'},
 {'tag': 'table',
  'xref_id': 'tbl3',
  'xref_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles//table/tbl3',
  'label': 'Table 3',
  'caption': 'Canine Frailty Index (CFI) and Health Related Quality of Life (HRQL) Scores across age-size groups with mean, standard deviation, interquartile range, range (minimum, maximum), and p-values for Kruskal-Wallis Tests showing significant differences across age-size groups',
  'footer': '',
  'media': [],
  'graphics': ['500746v1_tbl3', '500746v1_tbl3a'],
  'table_columns': [],
  'table_values': [],
  'xml': '',
  'id': '3.

In [25]:
df.loc[2, 'contents']

[{'tag': 'p',
  'text': 'Graphical summary of the design, results, and conclusions of the study.',
  'id': '2.2.1'},
 {'tag': 'fig',
  'caption': '',
  'xref_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles//figure/ufig1',
  'xref_id': 'ufig1',
  'label': '',
  'media': [],
  'graphics': ['500746v1_ufig1'],
  'pmcid': '',
  'id': '2.2.2'}]